In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
!pip install git+https://github.com/facebookresearch/fastText

  Cloning https://github.com/facebookresearch/fastText to /tmp/pip-req-build-kmy3cbr1
  Running command git clone -q https://github.com/facebookresearch/fastText /tmp/pip-req-build-kmy3cbr1
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3010123 sha256=808d5b70d79540781e8fa514453b8c80d7af99981933a224a35e3b90f77a065f
  Stored in directory: /tmp/pip-ephem-wheel-cache-w01ou5mq/wheels/72/f6/b9/3a859d37e24c73635e6c6af085ad99829e8b2ab1bd965b15a9
Successfully built fasttext


In [3]:
#%tensorflow_version 2.x

In [4]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [5]:
!ls /content/drive/My\ Drive/Code/data_aumentation_for_author_profiling

figures  notebooks  predictions  word_level_da


In [6]:
!ls /content/drive/My\ Drive/Data/depresion/2019/train

augmented		   original_prep
class_distpng		   original_prep_ground_truth.txt
golden_truth_filtered.txt  prep_chunks
golden_truth.txt	   prep_chunks_filtered_neg_only
ground_truth.txt	   train_golden_truth_joined.txt


In [7]:
import sys
sys.path.append('/content/drive/My Drive/Code/data_aumentation_for_author_profiling/')

In [8]:

from datetime import datetime
from word_level_da import utils
from word_level_da.preprocessing.load_data import Dataset

In [9]:
  
  utils.set_working_directory()
  epochs= 20      
  layers=1
  nodes=300
  dim=300
  label_pos=1
  len_doc=64
  key="anorexia18_drive"
  LOAD_EMB=False
  fast_file='cc.en.300.bin'
  

In [10]:
logger = utils.configure_root_logger(prefix_name="RNN"+key)

In [11]:
if LOAD_EMB:
  import fasttext.util
  fasttext.util.download_model('en', if_exists='ignore')  # English
  ft = fasttext.load_model('cc.en.300.bin')

In [12]:
from word_level_da.classifier.train_sequence_model import seq_model
import numpy as np

In [13]:
data = Dataset(key=key, doc_len=len_doc, min_len=int(len_doc / 2), chunking=True, remove_end=True)

In [14]:
training, test=data.get_train_test(return_ids=True)

In [15]:
bi_gru=seq_model(weights_path="/content/drive/My Drive/Code/datadata_aumentation_for_author_profiling/weights/", 
                 static=False, load_all_vectors=LOAD_EMB,
                 ids_labels=dict.fromkeys(test[2]).keys(), original_labels=test[3][0])

In [16]:
n_docs=[i for i in range(1,2)]
ratio=0
augmentation_method="Base"
model="cnn"
umbral=0.4
score_m="avg"
q=75

In [17]:
fold=0
n=0
#ratio=0
r=0.2

In [18]:
scores=[]
AUGMENTED=False

In [19]:
!ls "/content/drive/My Drive/Code/data_aumentation_for_author_profiling/word_level_da/obj/"

depresion19_local.pkl
pos_depresion19_local.pkl
predictions_depresion18_localcnn.csv
predictions_depresion18_localrnn-fixed.csv
predictions_depresion19_localcnn.csv
predictions_depresion19_localrnn-fixed.csv
r0_depresion19_local.pkl
r1_depresion19_local.pkl
WE_depresion18.pkl
WE_depresion19.pkl


In [20]:
for n in n_docs:
      prefix = augmentation_method + str(n)
      folder = augmentation_method + "/" + prefix
      truth_file = augmentation_method + "/" + prefix + ".txt"

      
      if AUGMENTED:
        docs, l_docs, ids, useless_data = data.get_dataset(folder_name=folder, truth_name=truth_file,
                                                               partition="augmented")
                
        new_training=np.append(training[0],docs)
        new_labels=np.append( training[1], l_docs)
      else:
        new_training=training[0]
        new_labels=training[1]

      info=bi_gru.build_model(((new_training, new_labels), (test[0], test[1])), layers=1, nodes=nodes, embedding_dim=dim, dropout_rate=r, TOP_K=150000, 
                                pretrained=True, embedding_trainable=False, bidirectional=True, kernel_size=[3,4,5],
                          seq_len=len_doc, emb_file=fast_file, class_imbanlance=True, algo=model,
                          vocab_dir="/content/drive/My Drive/Code/data_aumentation_for_author_profiling/word_level_da/obj",
                           key=key.split("_")[0])
      info[2]=umbral
      for i in range(1,3):
        score=bi_gru.train_model(1e-3, epochs=epochs, patience=6, batch_size=1024, load_weights=False, 
                                                weights_name=key.split("_")[0]+prefix+model+str(i)+".h5",
                                  ad_data=(test[3]), monitor_measure="val_loss", validation=True,
                                 umbral=umbral,  score_method=score_m, q=q)
        
        final_info=[augmentation_method,n,i ]
        final_info+=info+list(score)
        scores.append(final_info)
        print(final_info)
        logger.info(final_info)


Number of new embeddings:  0
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 64, 300)      31438200    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 62, 300)      270300      embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 61, 300)      360300      embedding[0][0]                  
__________________________________________________________

root              : INFO     ['Base', 1, 1, 104793, 150000, 0.4, 64, {0: 0.6239231446706417, 1: 2.5173794061185313}, 1.3949465260164366, 0.9195121951219513, 0.696969696969697, 0.2911392405063291, 0.41071428571428575, 0.05961067974567413, 0.9932909607887268, 0.3441614508628845, 0.8753674626350403, 0.6586376428604126, 0.6110484004020691, 0.5679638385772705, 0.5082619786262512, 0.437898188829422, 0.36953601241111755, 0.30687928199768066, 0.25478678941726685, 0.20900288224220276, 0.17025217413902283, 0.14077433943748474, 0.11736373603343964, 0.09836509078741074, 0.08252163231372833, 0.06988364458084106, 0.05961067974567413, 0.0, 0.0, 0.0, 0.0, 0.6304252743721008, 0.6711502075195312, 0.7049875855445862, 0.7536871433258057, 0.8055668473243713, 0.8479791879653931, 0.8856294751167297, 0.9141449928283691, 0.93730628490448, 0.9552999138832092, 0.9675173759460449, 0.9758976697921753, 0.9824363589286804, 0.9874985814094543, 0.9910194277763367, 0.9932909607887268, 0.0, 0.0, 0.0, 0.0, 0.516889333724

['Base', 1, 1, 104793, 150000, 0.4, 64, {0: 0.6239231446706417, 1: 2.5173794061185313}, 1.3949465260164366, 0.9195121951219513, 0.696969696969697, 0.2911392405063291, 0.41071428571428575, 0.05961067974567413, 0.9932909607887268, 0.3441614508628845, 0.8753674626350403, 0.6586376428604126, 0.6110484004020691, 0.5679638385772705, 0.5082619786262512, 0.437898188829422, 0.36953601241111755, 0.30687928199768066, 0.25478678941726685, 0.20900288224220276, 0.17025217413902283, 0.14077433943748474, 0.11736373603343964, 0.09836509078741074, 0.08252163231372833, 0.06988364458084106, 0.05961067974567413, 0.0, 0.0, 0.0, 0.0, 0.6304252743721008, 0.6711502075195312, 0.7049875855445862, 0.7536871433258057, 0.8055668473243713, 0.8479791879653931, 0.8856294751167297, 0.9141449928283691, 0.93730628490448, 0.9552999138832092, 0.9675173759460449, 0.9758976697921753, 0.9824363589286804, 0.9874985814094543, 0.9910194277763367, 0.9932909607887268, 0.0, 0.0, 0.0, 0.0, 0.5168893337249756, 0.5230954885482788, 0.5

root              : INFO     ['Base', 1, 2, 104793, 150000, 0.4, 64, {0: 0.6239231446706417, 1: 2.5173794061185313}, 1.3949465260164366, 0.9085365853658537, 0.527027027027027, 0.4936708860759494, 0.5098039215686275, 0.12039007991552353, 0.97511887550354, 0.3605844974517822, 0.863745927810669, 0.6604508757591248, 0.6130197048187256, 0.5731436014175415, 0.516392171382904, 0.44803258776664734, 0.37841618061065674, 0.31692585349082947, 0.26077619194984436, 0.21410831809043884, 0.17730151116847992, 0.14515738189220428, 0.12039007991552353, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6204630732536316, 0.6667856574058533, 0.7023023366928101, 0.7481706142425537, 0.7985251545906067, 0.8433306813240051, 0.8798614144325256, 0.9105511903762817, 0.934872567653656, 0.9509759545326233, 0.9649944305419922, 0.97511887550354, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5342011451721191, 0.46616023778915405, 0.5225433111190796, 0.39691224694252014, 0.3968051075935364, 0.3605844974517822, 0.4455004632472992, 

['Base', 1, 2, 104793, 150000, 0.4, 64, {0: 0.6239231446706417, 1: 2.5173794061185313}, 1.3949465260164366, 0.9085365853658537, 0.527027027027027, 0.4936708860759494, 0.5098039215686275, 0.12039007991552353, 0.97511887550354, 0.3605844974517822, 0.863745927810669, 0.6604508757591248, 0.6130197048187256, 0.5731436014175415, 0.516392171382904, 0.44803258776664734, 0.37841618061065674, 0.31692585349082947, 0.26077619194984436, 0.21410831809043884, 0.17730151116847992, 0.14515738189220428, 0.12039007991552353, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6204630732536316, 0.6667856574058533, 0.7023023366928101, 0.7481706142425537, 0.7985251545906067, 0.8433306813240051, 0.8798614144325256, 0.9105511903762817, 0.934872567653656, 0.9509759545326233, 0.9649944305419922, 0.97511887550354, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5342011451721191, 0.46616023778915405, 0.5225433111190796, 0.39691224694252014, 0.3968051075935364, 0.3605844974517822, 0.4455004632472992, 0.4417237639427185, 0.3748885

In [21]:
for score in scores:
  print(score)

['Base', 1, 1, 104793, 150000, 0.4, 64, {0: 0.6239231446706417, 1: 2.5173794061185313}, 1.3949465260164366, 0.9195121951219513, 0.696969696969697, 0.2911392405063291, 0.41071428571428575, 0.05961067974567413, 0.9932909607887268, 0.3441614508628845, 0.8753674626350403, 0.6586376428604126, 0.6110484004020691, 0.5679638385772705, 0.5082619786262512, 0.437898188829422, 0.36953601241111755, 0.30687928199768066, 0.25478678941726685, 0.20900288224220276, 0.17025217413902283, 0.14077433943748474, 0.11736373603343964, 0.09836509078741074, 0.08252163231372833, 0.06988364458084106, 0.05961067974567413, 0.0, 0.0, 0.0, 0.0, 0.6304252743721008, 0.6711502075195312, 0.7049875855445862, 0.7536871433258057, 0.8055668473243713, 0.8479791879653931, 0.8856294751167297, 0.9141449928283691, 0.93730628490448, 0.9552999138832092, 0.9675173759460449, 0.9758976697921753, 0.9824363589286804, 0.9874985814094543, 0.9910194277763367, 0.9932909607887268, 0.0, 0.0, 0.0, 0.0, 0.5168893337249756, 0.5230954885482788, 0.5

In [22]:
bi_gru.save_predictions("/content/drive/My Drive/Code/data_aumentation_for_author_profiling/predictions/", "predictions_"+model+"_"+augmentation_method)    
logger.info("Finish Time: %s", datetime.now())

root              : INFO     Finish Time: 2020-08-19 03:27:16.309587
